In [1]:
import pandas as pd

#data
df_stations = pd.read_csv('station_emissions.csv')

#Assumptions:
ei = 0.7 #HELE coal plant has emission intensity of 700 g/kWh (0.7 tonnes/MWh)
cf = 0.85 #capacity factor new coal
cost = 3100 #Cost of new installed capacity (from APGTR, $3,100, USC black coal)

In [2]:
df_fuel = df_stations.groupby('CATEGORY').sum()[['GWH','EMISSIONS']]
df_fuel

,GWH,EMISSIONS
CATEGORY,,
Biomass,577.650409,NaN
Black Coal,104078.102940,90996.559285
Brown Coal,47021.910707,58104.190085
Distillate,133.722916,64.450031
Gas,23949.254238,11314.359856
Hydro,25484.696914,NaN
Solar,481.955180,NaN
Wind,12382.680455,NaN


In [3]:
df_coal = df_fuel.ix[['Brown Coal','Black Coal']]
df_coal

,GWH,EMISSIONS
CATEGORY,,
Brown Coal,47021.910707,58104.190085
Black Coal,104078.102940,90996.559285


In [4]:
print df_coal.sum()

GWH          151100.013647
EMISSIONS    149100.749371
dtype: float64


In [7]:
#Emissions caculation
total_emissions_2016 = df_fuel.sum().EMISSIONS
coal_emissions_2016 = df_coal.sum().EMISSIONS
coal_output_2016 = df_coal.sum().GWH

#Assume new HELE coal plant has emission intensity of 700 g/kWh (0.7 tonnes/MWh)
hele_emissions = ei * coal_output_2016
emissions_decrease = coal_emissions_2016 - hele_emissions


print "Absolute decrease: {0:.2f} MT".format(emissions_decrease/1000)
print "Percentage decrease: {0:.2f}% ".format(emissions_decrease/total_emissions_2016*100)

Absolute decrease: 43.33 MT
Precentage decrease: 27.00% 


In [6]:
#Capacity and Cost calculation

#Capacity factor of new coal plant
new_capacity = coal_output_2016 / (cf * 8760)
print "HELE Capacity require: {0:.2f} GW".format(new_capacity)

#Cost of new installed capacity (from Australian Power Generation Technology Report, $3,100, USC black coal)
total_capex = new_capacity * cost
print "Total cost: ${0:.2f} billion".format(total_capex/1000)

HELE Capacity require: 20.29 GW
Total cost: $62.91 billion
